In [1]:
import os,sys
from src.Mask.Utils import SaveModel,read_yaml,create_directory
from src.Mask.exceptions import CustomException
from src.Mask.loggers import logger
from src.Mask.Constants import *
from dataclasses import dataclass
from pathlib import Path

In [2]:
#step3:- update the entity file:entity file is nothing but whatever parameter we r defining in yaml file 
#in entity class we defined them as class varibale along with dtype
@dataclass
class BaseModelConfig():
    root_dirpath:Path
    base_model_path:Path
    updated_base_model_path:Path
    all_param:dict


In [3]:
#step4:update the configurationmanager file :In this file we are reading 
#the yaml file and creating directory and also assigning the value to class variable and returning them as fucntn
class ConfigurationManager():
    #constructor method to initialize the instance variable init
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        self.config = read_yaml(config_filepath) #rtn value as configbox dictatonary
        self.param = read_yaml(param_filepath)   #rtn value as configbox dictatonary

        #creating main artifact directory
        create_directory([self.config.artifact_root_dir]) #create artifacts folder 

    #creating another method to create base model directory
    def get_base_model_config(self) ->BaseModelConfig:
        #allocating local variables used inside this method
        config = self.config.prepare_base_model
        param = self.param

        #creating directory base_model
        create_directory([config.root_dirpath])

        #creating an object of class variable and assigning value to class varibale
        base_model_config = BaseModelConfig(
            root_dirpath=config.root_dirpath,
            base_model_path=config.base_model_path,
            updated_base_model_path=config.updated_base_model_path,
            all_param=param
        )
        return base_model_config


In [4]:
import tensorflow as tf

In [5]:
#step5)update the component file : in this file we create object of class variable and do task accordingly
class BaseModel():
    def __init__(self,modelconfig:BaseModelConfig):
        self.modelconfig = modelconfig

    #creating method to load the vgg16 base pretrained model and download (weighs and bias parameter init)
    def basemodel(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.modelconfig.all_param.include_top,
            weights=self.modelconfig.all_param.weights,
            input_shape=self.modelconfig.all_param.input_shape,
            classes=self.modelconfig.all_param.classes,
            classifier_activation=self.modelconfig.all_param.classifier_activation
        )
        # To see the summary of the base model
        logger.info(f"Base model Summary: \n{self.model.summary()}")

        #saving the base model files in artifacts directory
        SaveModel(filepath=Path(self.modelconfig.base_model_path),model=self.model)

    @staticmethod
    def _prepare_full_Sequentialmodel(basemodel,freeze,freeze_till,learning_rate,classes):
        #creating a sequential model object
        sequential_model = tf.keras.models.Sequential()

        #now adding all the layers of base model to this sequential model object
        for layer in basemodel.layers:
            sequential_model.add(layer)

        #now freezing all the layer from sequential model except the dense layers
        if freeze:
            for layer in sequential_model.layers:
                layer.trainable = False #this line indicate we r using pretrained weights and bias for the custom dataset no need to change the parameter value during training

        elif (freeze_till is not None) and (freeze_till>0):
            for layer in sequential_model.layers[:freeze_till]: #remove output layer from sequemntial model
                layer.trainable = False

        #now adding flatten layer to the model
        sequential_model.add(tf.keras.layers.Flatten())

        #now adding customized Output layer to this model
        sequential_model.add(tf.keras.layers.Dense(units=classes,activation="sigmoid"))

        logger.info(f"Customizes Sequential Model: {sequential_model.summary()}")

        #now compiling the sequential model
        sequential_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss = "binary_crossentropy",
            metrics = ["accuracy"]
        )
        return sequential_model


    #Now Updating the base Model By Creating Custom Sequential Model
    def update_base_model(self):
        #creating an object for sequential model
        self.sequential_model = self._prepare_full_Sequentialmodel(
            basemodel = self.model,
            freeze = True,
            freeze_till = None,
            learning_rate = self.modelconfig.all_param.learning_rate,
            classes = self.modelconfig.all_param.classes,
            
        )

        #saving the sequential model
        SaveModel(filepath=Path(self.modelconfig.updated_base_model_path),model = self.sequential_model)

In [6]:
os.chdir('../')
%pwd

'd:\\MaskImageDetection\\MaskDetectionModel'

In [7]:
#step6) update the pipeline to perform operation
try:
    cm = ConfigurationManager()
    base_model_config = cm.get_base_model_config()

    #creating an object of basemodel
    bm = BaseModel(base_model_config)
    bm.basemodel()

    bm.update_base_model()

except Exception as e:
    raise CustomException(e,sys)

[2024-09-30 15:25:22,187]-INFO-33-Yaml file config\config.yaml reading
[2024-09-30 15:25:22,227]-INFO-33-Yaml file param.yaml reading
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     1475

d:\MaskImageDetection\MaskDetectionModel\Mask\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2024-09-30 15:25:26,702]-INFO-58-Model saved at artifacts\base_model\basemodel.h5
[2024-09-30 15:25:27,041]-INFO-45-Customizes Sequential Model: <keras.src.engine.sequential.Sequential object at 0x0000029732A784C0>
[2024-09-30 15:25:27,576]-INFO-58-Model saved at artifacts\base_model\updated_model.h5
